<a href="https://colab.research.google.com/github/aicuai/Book-SD-MasterGuide/blob/main/ComfyJapan_LTX2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ComfyJapan] ComfyUI LTX-2 動画生成

## 🚀 概要

Lightricks社の最新動画生成モデル **LTX-2** を Google Colab で運用するための統合環境です。ポーズ制御（Pose to Video） と テキスト生成（Text to Video） の両ワークフローに完全対応し、24時間稼働を想定した「自己修復機能」を備えています。

### 🛠️ 技術解説と推奨環境

* **推奨GPU**: **L4 GPU (22.5GB)** 以上。生成速度と安定性の面から **A100 / H100** を強く推奨します。

* **メモリ最適化**: L4/T4 GPUではVRAM容量がモデルサイズ（19B + 12B）に対してシビアなため、自動的にメモリ節約モード（`--lowvram`）を適用します。

参考

| GPU | RAM | VRAM | Storage | 備考 |
|-----|-----|------|---------|------|
| T4 | 12GB | 15GB | 112GB | 動作可能（制限あり） |
| L4 | 53GB | 22.5GB | 112GB | 推奨最小構成 |
| A100/H100 | 83GB | 40GB+ | 112GB | 強く推奨 |



* **自己修復ロジック**:
* **モデル配置**: Gemma 3IT 等の複雑な階層モデルを ComfyUI 認識ディレクトリへ自動整理します。
* **自動再起動**: サーバーがメモリ不足（OOM）などでクラッシュした場合、スレッドがそれを検知して Discord に報告し、自動的にプロセスを再立ち上げします。


### 📋 事前準備

1. **HuggingFace**: [LTX-2 Repository](https://huggingface.co/Lightricks/LTX-2) で利用許諾を済ませたアカウントの `HF_TOKEN` を、Colabの「🔑シークレット」に設定してください。
2. **Discord**: 通知用 Webhook URL を `DISCORD_URL` として設定してください。

---


# 🥚各種モデルのDL ＆ ComfyUIの起動🐣
##＜はじめる前の準備＞
### ❶ Google Colab の「シークレット」に `HF_TOKEN` と `DISCORD_URL` を設定してください。
### ❷（▶）ボタンをクリックして起動。完了すると Discord に通知が届きます。


添付の公式ワークフローを読み込む際にモデルが見つかりません

t2vグラフを読み込む際に、次のモデルが見つかりませんでした

checkpoints / ltx-2-19b-distilled.safetensors

latent_upscale_models / ltx-2-spatial-upscaler-x2-1.0.safetensors

loras / ltx-2-19b-lora-camera-control-dolly-left.safetensors

i2vグラフを読み込む際に、次のモデルが見つかりませんでした

checkpoints / ltx-2-19b-distilled.safetensors

latent_upscale_models / ltx-2-spatial-upscaler-x2-1.0.safetensors

loras / ltx-2-19b-lora-camera-control-dolly-left.safetensors

📂 ComfyUI/
└── 📂 models/
    ├── 📂 checkpoints/
    │      └── ltx-2-19b-distilled.safetensors
    ├── 📂 text_encoders/
    │      └── gemma_3_12B_it.safetensors
    ├── 📂 loras/
    │      └── ltx-2-19b-lora-camera-control-dolly-left.safetensors
    └── 📂 latent_upscale_models/
           └── ltx-2-spatial-upscaler-x2-1.0.safetensors



In [ ]:
# @markdown # 👈 最終統合ボタン（エラー回避・自動修復機能付）
# @markdown ---
# @markdown ※ Secrets設定に失敗しても、ComfyUIは起動するように修正しました。

import os, socket, time, threading, requests, shutil, subprocess, re, sys
from collections import deque
from google.colab import userdata, output
from IPython.display import clear_output

# ==========================================
# 🔑 1. 設定 & 通知機能 (エラー回避型)
# ==========================================
def get_colab_secret(key):
    try:
        return userdata.get(key)
    except:
        print(f"⚠️ 警告: シークレット '{key}' を取得できません。Secretsタブで許可を確認してください。")
        return None

webhook_url = get_colab_secret('DISCORD_URL')
hf_token = get_colab_secret('HF_TOKEN')
if hf_token: os.environ['HF_TOKEN'] = hf_token

def send_discord(msg):
    if webhook_url:
        try: requests.post(webhook_url, json={"content": msg}, timeout=10)
        except: pass

# GPU情報の取得
try:
    gpu_full = subprocess.check_output(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"]).decode().strip()
    gpu_name = gpu_full.split(",")[0]
except: gpu_full = gpu_name = "GPU"

# 🚀 起動通知
send_discord(f"🛡️ **LTX-2 防衛モード起動**\n💻 GPU: `{gpu_full}`\n💾 空き容量: `{shutil.disk_usage('/')[2] // (2**30)}GB`")

# ==========================================
# 📁 2. 環境構築 & 整合性チェック
# ==========================================
%cd /content
if not os.path.exists("ComfyUI/main.py"):
    print("📥 ComfyUI をインストール中...")
    subprocess.run(["git", "clone", "https://github.com/Comfy-Org/ComfyUI.git", "-q"])

print("🛠️ ライブラリを点検中...")
!pip install -q torchsde==0.2.6 onnxruntime-gpu comfy-cli requests spandrel av albumentations opencv-python
%cd /content/ComfyUI
!pip install -q -r requirements.txt

# ==========================================
# 📥 3. 絶対に外さない「必須4ファイル」
# ==========================================
print("🛡️ 必須モデルの存在を確認中...")

def safe_download(repo, file, folder, size_gb):
    dest = os.path.join("/content/ComfyUI", folder, file)
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    if not os.path.exists(dest):
        free = shutil.disk_usage('/')[2] // (2**30)
        if free > (size_gb + 2):
            print(f"📦 ダウンロード開始({size_gb}GB): {file}")
            subprocess.run(["hf", "download", repo, file, "--local-dir", os.path.join("/content/ComfyUI", folder), "--quiet"])
        else:
            print(f"🚨 スキップ: {file} (残り容量 {free}GB 不足)")

# 必須①：LTX-2 FP8 本体 (19GB)
safe_download("Lightricks/LTX-2", "ltx-2-19b-dev-fp8.safetensors", "models/checkpoints", 19)

# 必須②：Gemma 3 IT (12GB) [cite: 66, 202]
safe_download("Comfy-Org/ltx-2", "split_files/text_encoders/gemma_3_12B_it.safetensors", "models/text_encoders", 12)
g_path = "models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors"
if os.path.exists(g_path): shutil.move(g_path, "models/text_encoders/gemma_3_12B_it.safetensors")

# 必須③：Spatial Upscaler (1GB) [cite: 291, 292]
safe_download("Lightricks/LTX-2", "ltx-2-spatial-upscaler-x2-1.0.safetensors", "models/latent_upscale_models", 1)

# 必須④：Distilled LoRA (1GB)
safe_download("Lightricks/LTX-2", "ltx-2-19b-distilled-lora-384.safetensors", "models/loras", 1)

# カスタムノードの最終点検
%cd /content/ComfyUI/custom_nodes
for url in ["https://github.com/Comfy-Org/ComfyUI-Manager", "https://github.com/Lightricks/ComfyUI-LTXVideo"]:
    name = url.split("/")[-1]
    if not os.path.exists(name): subprocess.run(["git", "clone", url, "-q"])

# ==========================================
# 📡 4. URL発行 & ログ監視
# ==========================================
def monitor_ready(port):
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            if sock.connect_ex(('127.0.0.1', port)) == 0:
                url = output.eval_js(f'google.colab.kernel.proxyPort({port})')
                msg = f"🎉 **ComfyUI Ready!**\n💻 GPU: `{gpu_name}`\n🔗 URL: {url}"
                print(f"\n{'★'*50}\n{msg}\n{'★'*50}\n")
                send_discord(msg)
                break
            sock.close()
        except: pass
        time.sleep(5)

threading.Thread(target=monitor_ready, args=(8188,), daemon=True).start()

# ==========================================
# 🚀 5. 起動 (H100/A100 最速設定)
# ==========================================
%cd /content/ComfyUI
mem_flags = "" if any(x in gpu_name for x in ["H100", "A100"]) else "--lowvram"
print(f"🚀 {gpu_name} 最速モードで起動します...")
!python main.py {mem_flags} --cache-none --dont-print-server

In [ ]:
import os, subprocess, shutil

# 配置先の定義
LORA_DIR = "/content/ComfyUI/models/loras"
os.makedirs(LORA_DIR, exist_ok=True)

# ターゲット情報
repo = "Lightricks/LTX-2-19b-LoRA-Camera-Control-Dolly-Left"
file = "ltx-2-19b-lora-camera-control-dolly-left.safetensors"
dest_path = os.path.join(LORA_DIR, file)

def final_piece_download():
    if not os.path.exists(dest_path):
        print(f"📥 最後のピースをダウンロード中: {file}...")
        # 直接ダウンロード
        subprocess.run(["hf", "download", repo, file, "--local-dir", LORA_DIR, "--quiet"])
        print(f"✅ 配置完了: {dest_path}")
    else:
        print(f"✅ すでに存在します: {file}")

    print("\n✨ これですべてのファイルが揃いました！")
    print("🚀 ComfyUI の画面で「Refresh」ボタンを押して、生成を開始してください。")

final_piece_download()

In [ ]:
📊 現在のディスク空き容量: 18GB
❌ 容量不足: 残り18GBではダウンロードできません。
💡 対策: 他の巨大な.safetensors（dev-fp8版など）を削除して空きを作ってください。

In [ ]:
import os, shutil, subprocess

# ==========================================
# 🧹 ディスク容量確保ロジック
# ==========================================
def swap_models_for_distilled():
    base_path = "/content/ComfyUI/models"
    dev_model = os.path.join(base_path, "checkpoints/ltx-2-19b-dev-fp8.safetensors")
    distilled_model = os.path.join(base_path, "checkpoints/ltx-2-19b-distilled.safetensors")

    # 1. 蒸留版がない場合、Dev版を削除して容量を作る
    if not os.path.exists(distilled_model):
        if os.path.exists(dev_model):
            print(f"🧹 容量確保のため、標準版を削除します: {dev_model}")
            os.remove(dev_model)
        else:
            print("💡 標準版モデルは既に削除されているか、存在しません。")

    # 2. 不要なキャッシュの強制排除
    !pip cache purge

    total, used, free = shutil.disk_usage("/")
    free_gb = free // (2**30)
    print(f"📊 現在の空き容量: {free_gb}GB")

    # 3. 不足分の再ダウンロード（容量チェック付）
    def safe_download(repo, file, folder, min_gb):
        dest = os.path.join(folder, file)
        os.makedirs(folder, exist_ok=True)
        if not os.path.exists(dest):
            _, _, current_free = shutil.disk_usage("/")
            if (current_free // (2**30)) > min_gb:
                print(f"📥 ダウンロード中: {file}...")
                subprocess.run(["hf", "download", repo, file, "--local-dir", folder, "--quiet"])
                print(f"✅ 完了: {file}")
            else:
                print(f"❌ 容量不足のためスキップ: {file} (残り {current_free // (2**30)}GB)")

    # 蒸留版本体 (約35GB)
    safe_download("Lightricks/LTX-2", "ltx-2-19b-distilled.safetensors", os.path.join(base_path, "checkpoints"), 36)
    # アップスケーラー
    safe_download("Lightricks/LTX-2", "ltx-2-spatial-upscaler-x2-1.0.safetensors", os.path.join(base_path, "latent_upscale_models"), 2)
    # カメラLoRA
    safe_download("Lightricks/LTX-2-19b-LoRA-Camera-Control-Dolly-Left", "ltx-2-19b-lora-camera-control-dolly-left.safetensors", os.path.join(base_path, "loras"), 1)

# 実行
swap_models_for_distilled()

In [ ]:
# 🚨 警告: 失敗した巨大なダウンロード残骸を消去してディスクを「70GB以上」空けます
!rm -rf /content/ComfyUI/models/checkpoints/*.safetensors
!rm -rf /root/.cache/huggingface
!pip cache purge
import shutil
print(f"📊 掃除後の空き容量: {shutil.disk_usage('/')[2] // (2**30)}GB")

In [ ]:
# @markdown # 🚀 LTX-2 FP8版 起動セル（ディスク・URL表示 修正版）

import os, socket, time, threading, requests, shutil, subprocess, re, sys
from google.colab import userdata, output
from IPython.display import clear_output

# --- 設定 ---
hf_token = userdata.get('HF_TOKEN')
webhook_url = userdata.get('DISCORD_URL')
if hf_token: os.environ['HF_TOKEN'] = hf_token

def send_discord(msg):
    if webhook_url:
        try: requests.post(webhook_url, json={"content": msg}, timeout=10)
        except: pass

# --- URL表示を最優先するスレッド ---
def monitor_url(port):
    print(f"📡 接続待ち... (Port {port})")
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            res = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if res == 0:
                url = output.eval_js(f'google.colab.kernel.proxyPort({port})')
                msg = f"🎉 **ComfyUI 起動完了**\n🔗 URL: {url}"
                print(f"\n{'★'*40}\n{msg}\n{'★'*40}\n")
                send_discord(msg)
                break
        except: pass
        time.sleep(2)

# --- 1. ディスク確認 & 最小ダウンロード ---
%cd /content
if not os.path.exists("ComfyUI"):
    !git clone https://github.com/Comfy-Org/ComfyUI.git -q

# 必須モデルのみ（FP8版=19GB）
print("📥 最小限のモデル(FP8)を確認中...")
def hf_get(repo, file, folder):
    dest = os.path.join("/content/ComfyUI", folder, file)
    if not os.path.exists(dest):
        !hf download {repo} {file} --local-dir {"/content/ComfyUI/"+folder} --quiet

# Gemma(12GB) + LTX2-FP8(19GB) = 計31GB
hf_get("Comfy-Org/ltx-2", "split_files/text_encoders/gemma_3_12B_it.safetensors", "models/text_encoders")
g_wrong = "/content/ComfyUI/models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors"
if os.path.exists(g_wrong): shutil.move(g_wrong, "/content/ComfyUI/models/text_encoders/gemma_3_12B_it.safetensors")

hf_get("Lightricks/LTX-2", "ltx-2-19b-dev-fp8.safetensors", "models/checkpoints")

# --- 2. サーバー起動 ---
threading.Thread(target=monitor_url, args=(8188,), daemon=True).start()
%cd /content/ComfyUI
print("🚀 サーバーを立ち上げます...")
!python main.py --cache-none --dont-print-server

# 技術的な改善点まとめ
## 技術解説：2026年版 LTX-2 運用ガイド

この環境では、以下の高度な最適化が施されています。

VRAM マネジメント: A100 80GB ではモデル全体を VRAM に保持する高速推論を、L4 22.5GB では --lowvram を用いた堅実な動作を自動的に選択します。

アセット不整合の解消: ワークフローが要求する woman_with_paper 系のファイルを、現在の正しい配布URLから自動取得します。

マルチリポジトリ・ダウンロード: 単一の配布元では揃わない Pose Control LoRA や Text Encoder を、複数の Hugging Face リポジトリを横断して一括で揃えます。

これで、準備は万全です。提供された JSON ファイル を読み込み、LTX-2 の驚異的な動画生成能力をご体験ください！


ディスク容量の「見える化」: 準備完了通知に現在のディスク残量を記載するようにしました。これにより、あとどれくらいモデルを追加できるか一目で分かります。

フェイルセーフダウンロード: safe_hf_get 関数が、Google Colabのディスク残量を1ファイルごとにチェックします。もし 35GB の Distilled モデルを入れようとして残りが 20GB しかなければ、ダウンロードをスキップしてDiscordに警告を送ります。

クラッシュ時の自己診断: ログに FileNotFoundError (一時ファイル作成不可) が出た場合、今回のコードは「ディスクパンク」と即座に判定してDiscordに報告します。

これで「いつの間にかディスクがいっぱいで動かなくなっていた」という事態を未然に防げるようになります。まずは実行して、残量が何GBと表示されるか確認してみてください


# 謝辞：このノートブックは AICU Japan がメンテナンスしております。質問は [X@AICUai](https://x.com/AICUai) まで。